In [1]:
%reset -f
%load_ext autoreload
%autoreload 2

import Odin as backend

# validation test for incompressible flow

geom = backend.createGeometry( 'rectangle', [4,1] )
mesh = backend.createMesh( geom, res=1 )

# make just a list, give a name as optional parameter to flow model
myFlowModels = [
    backend.FlowModels.ConstPresGrad(depVariableName='p'),
    backend.FlowModels.IncompressibleMomentumComp(depVariableName='u', orientation=0, otherVelocityFieldName='v', pressureFieldName='p', kinViscosityName='nu')
]

sim1 = backend.Simulation(myFlowModels, mesh, geom )

myFields = sim1.getFieldRegistry()
print(myFields)

myFields['nu'] = 1.0

myFlowModels[0].initializeFields(myFields, mesh)
myFlowModels[0].setConstPresGrad(-0.0, myFields)

p = myFields['p']
print(p.data)

U = myFields['u']
#U.data.fill(2.0)

U.defineBoundaryCondition(boundaryName='top', boundaryType='zeroGradient')
U.defineBoundaryCondition(boundaryName='bottom', boundaryType='zeroGradient')
U.defineBoundaryCondition(boundaryName='left', boundaryType='fixedValue', value=1 )
#U.defineBoundaryCondition(boundaryName='left', boundaryType='zeroGradient')
U.defineBoundaryCondition(boundaryName='right', boundaryType='zeroGradient')

print(U.data)

sim1.solve('u')
print(U.data)

sim1.display(p, mesh)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
{'governor': <Fields.fieldGovernor object at 0x7fdc597e41f0>, 'invCellDist': <Fields.fieldContainer object at 0x7fdc222f76d0>, 'u': <BaseField.baseField object at 0x7fdc222f79d0>, 'v': <BaseField.baseField object at 0x7fdc222f7a00>, 'p': <BaseField.baseField object at 0x7fdc222f7a30>, 'nu': 0.0}
[[0. 0. 0. 0.]]
[[1. 0. 0. 0. 0.]]


In [ ]:
# shutting the interpreter down, so I can a fresh instance next time.
# my modules are automatically checked for updates
import os
os._exit(00)
